In [4]:
import xarray as xr
import matplotlib.pyplot as plt
from datetime import date


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
def get_today_date():
    return date.today().strftime("%Y%m%d")

# Example usage:
today_date = get_today_date()
print(today_date)


20241216


In [3]:
url=f"http://nomads.ncep.noaa.gov:80/dods/gfs_0p25_1hr/gfs{today_date}/gfs_0p25_1hr_06z"
ds =xr.open_dataset(url).sel(lon=slice(60,100),lat=slice(0,40))

OSError: [Errno -70] NetCDF: DAP server error: 'http://nomads.ncep.noaa.gov:80/dods/gfs_0p25_1hr/gfs20241216/gfs_0p25_1hr_06z'

In [6]:
sfcx=ds['apcpsfc'].sel(lon=slice(60,100),lat=slice(0,40))#.resample(time='3H').mean()
dt = sfcx.time
sfcx1=sfcx.fillna(0)
sfcx=sfcx1.diff(dim='time').dropna(dim='time')


temp=ds['tmpsfc'].fillna(0)
mslp=ds['pressfc'].fillna(0)/100
cloud=ds['tcdcaveclm'].fillna(0)


In [8]:

def dict_l(data, fm):
    # Convert the data to a dictionary
    data_dict = data.squeeze().to_dict(data='list', encoding=False)

    def convert_types(obj):
        if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
            return obj.isoformat()  # Convert datetime objects to ISO format
        elif isinstance(obj, timedelta):
            return obj.total_seconds()  # Convert timedelta to total seconds
        elif isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert numpy arrays to lists
        elif isinstance(obj, (np.float32, np.float64)):
            return float(obj)  # Convert numpy floats to native floats
        elif isinstance(obj, (np.int32, np.int64)):
            return int(obj)  # Convert numpy integers to native ints
        raise TypeError(f"Type {type(obj)} not serializable")

    file = f"{fm}.json"
    # Save the dictionary as a JSON file with the extended convert_types function
    with open(file, 'w') as json_file:
        json.dump(data_dict, json_file, indent=4, default=convert_types)

# Example usage (make sure to replace this with your actual data)
# df = pd.DataFrame(...)  # Your DataFrame here
# dict_l(df, 'output_filename')


In [49]:
dict_l(sfcx,"rf_tot_nasa")
dict_l(temp,"temp_tot_nasa")
dict_l(mslp,"mslp_tot_nasa")
dict_l(cloud,"cloud_tot_nasa")
